In [1]:
%matplotlib widget

import rospy
import os
import threading
import time #wait
import sys # standard

import actionlib
import actionlib.msg
# import assignment_2_2022.msg
from assignment_2_2022.msg import PlanningAction
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2022.srv import *
from geometry_msgs.msg import Twist, Point, Pose
from assignment_2_2022.msg import *

import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:
global goal_reached
global goal_cancelled
goal_reached = 0
goal_cancelled = 0

class Node_A:
    
    def __init__(self):
        
        # variables
        self.pos_x = 0.0
        self.pos_y = 0.0
        self.goal_x = 0.0
        self.goal_y = 0.0
        
        self.count_g = 0
        
        # trajectory
        self.data_x = []
        self.data_y = []
        
        self.goal_his_x = []
        self.goal_his_y = []
        
        # moving:True, waiting:False
        self.robot_state = False 
        
        # initializes figure and axis for map
        self.fig, self.ax = plt.subplots()
        self.map_rob, = plt.plot([],[], marker='o',
                                 markersize=12,
                                 markerfacecolor='b',
                                 label='Robot')
        self.map_goal_cur, = plt.plot([],[], marker='D',
                                 markersize=12,
                                 markerfacecolor='g',
                                 label='Current goal')
        self.map_goal_his, = plt.plot([],[], marker='X',
                                 markersize=12,
                                 linewidth=0,
                                 markerfacecolor='r',
                                 label='Reached/cancelled goal')

        
        # initializes pub and sub
        self.pub = rospy.Publisher('/robot_data', Info, queue_size = 1)
        self.sub = rospy.Subscriber('/odom', Odometry, self.Info_Callback)
        
        # waits for service server
        rospy.wait_for_service('goal_info')      
        # initializes service client
        self.srv = rospy.ServiceProxy('goal_info', Goal)
        
    def map_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.set_title('Positions of the robot and goals')
        self.ax.set_xlabel('x [m]')
        self.ax.set_ylabel('y [m]')
        return self.map_rob, self.map_goal_cur, self.map_goal_his
    
    def map_update(self, frame):
        self.map_rob.set_data(self.data_x, self.data_y)
        self.map_goal_cur.set_data(self.goal_x, self.goal_y)
        self.map_goal_his.set_data(self.goal_his_x, self.goal_his_y)
        return self.map_rob, self.map_goal_cur, self.map_goal_his
        
    def goal_checker(self):
        while not rospy.is_shutdown():
            goals = self.srv(0)
            goal_reached = goals.total_r
            goal_cancelled = goals.total_c
            time.sleep(1)
    
    def Actionclient(self):
        # initializes action client
        self.client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
        # waits for server's running
        rospy.loginfo('Waiting for server.')
        self.client.wait_for_server()
        # creates a goal to send
        self.goal = assignment_2_2022.msg.PlanningGoal()
        rospy.spin()
    
    def Info_Callback(self, msg):
        self.pos_x = msg.pose.pose.position.x
        self.pos_y = msg.pose.pose.position.y
        self.data_x.append(self.pos_x)
        self.data_y.append(self.pos_y)
        info = Info()
        info.x = msg.pose.pose.position.x
        info.y = msg.pose.pose.position.y
        info.vel_x = msg.twist.twist.linear.x
        info.vel_y = msg.twist.twist.linear.y
        self.pub.publish(info)
            
    def Interface(self):
        # message for user
        rospy.loginfo('This is the user interface.')
        
        # sliders to set new goal position
        self.new_goal_x = widgets.FloatSlider(
            value=0.0,
            min=-10,
            max=10.0,
            step=0.1,
            description='New x:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )
        
        self.new_goal_y = widgets.FloatSlider(
            value=0.0,
            min=-10,
            max=10.0,
            step=0.1,
            description='New y:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )
        
        # text boxes for setting new goal
        self.new_goal_X = widgets.FloatText()
        self.new_goal_Y = widgets.FloatText()
        
        # buttons to send/cancel goal
        self.button_new = Button(
            description='Send new goal',
            layout=Layout(width='auto',align="center",grid_area='new'),
            style=ButtonStyle(button_color='mediumaquamarine')
        )
        
        self.button_cancel = Button(
            description='Cancel current goal',
            layout=Layout(width='auto', grid_area='cancel'),
            style=ButtonStyle(button_color='lightsalmon')
        )
        
        # values for buttons
        self.button_new.value = 1
        self.button_cancel.value = 2
        
        # output for widgets
        self.output = widgets.Output()
        
        # on_click functions
        self.button_new.on_click(self.on_button_clicked)
        self.button_cancel.on_click(self.on_button_clicked)

        
        box1 = widgets.HBox([self.new_goal_x, self.new_goal_X])
        box2 = widgets.HBox([self.new_goal_y, self.new_goal_Y])
        box3 = widgets.HBox([self.button_new, self.button_cancel])
        
        mylink1 = widgets.jslink((self.new_goal_x, 'value'), (self.new_goal_X, 'value'))
        mylink2 = widgets.jslink((self.new_goal_y, 'value'), (self.new_goal_Y, 'value'))
        
        display(box1)
        display(box2)
        display(box3)
        display(self.output)
    
    def on_button_clicked(self,b):
        self.output.clear_output(True)
        with self.output:
            if b.value == 1:
                if self.count_g != 0:
                    self.goal_his_x.append(self.goal_x)
                    self.goal_his_y.append(self.goal_y)
                self.goal.target_pose.pose.position.x = float(self.new_goal_x.value)
                self.goal.target_pose.pose.position.y = float(self.new_goal_y.value)
                self.client.send_goal(self.goal)
                rospy.loginfo('The new goal was sent.')
                self.goal_x = self.new_goal_x.value
                self.goal_y = self.new_goal_y.value
                self.robot_state = True
                self.count_g += 1
            elif b.value == 2:
                if self.robot_state == True and (abs(self.goal_x-self.pos_x) > 1 or abs(self.goal_y-self.pos_y) > 1):
                    self.client.cancel_goal()
                    rospy.loginfo('The current goal was cancelled.')
                    resp = self.srv(1)
                    self.goal_his_x.append(self.goal_x)
                    self.goal_his_y.append(self.goal_y)
                self.robot_state = False
    

        
rospy.init_node('Node_A')
node_a = Node_A()

node_a.Interface()

# threads
thread1 = threading.Thread(target=node_a.Actionclient)
thread2 = threading.Thread(target=node_a.goal_checker)
thread1.start()
thread2.start()

map_ani = FuncAnimation(node_a.fig, node_a.map_update, init_func=node_a.map_init, cache_frame_data=False)

plt.legend()
plt.show(block=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[INFO] [1684664275.831533, 1632.665000]: This is the user interface.


Output()

[INFO] [1684664276.066488, 1632.880000]: Waiting for server.


In [3]:
global closest_obs
def Laser_Callback(msg):
    closest_obs.value = min(msg.ranges)
    # rospy.loginfo(closest_obs.value)

sub_laser = rospy.Subscriber('/scan', LaserScan, Laser_Callback)
closest_obs = widgets.FloatText(
    value=0.0,
    description='Closest obstacle:',
    disabled=False,
    continuous_update=True,
    readout_format='.3f',
)
display(closest_obs)

FloatText(value=0.0, continuous_update=True, description='Closest obstacle:')